In [12]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import cross_validation
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LassoLarsCV
import sklearn
import pandas as pd

%matplotlib inline

In [3]:
df = pd.read_csv("../results/melted_hackseq.csv")
df

,GC count,Amino Acid Cut position,NGGX_pd.Order2_P0,NGGX_pd.Order2_P1,NGGX_pd.Order2_P10,NGGX_pd.Order2_P11,NGGX_pd.Order2_P12,NGGX_pd.Order2_P13,NGGX_pd.Order2_P14,NGGX_pd.Order2_P15,...,Target gene,Tm global_False,5mer_end_False,8mer_middle_False,5mer_start_False,GC < 10,Transcript,Strand,drug,lfc
0,13.0,254.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,CCDC101,71.954952,-23.661612,14.686540,-33.479130,0,ENST00000317058,antisense,AZD_200nM,2.896417
1,14.0,216.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,CCDC101,70.136401,-31.269027,15.830283,-31.374070,0,ENST00000317058,antisense,AZD_200nM,2.867987
2,10.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,64.729741,-29.148801,6.039043,-31.843549,0,ENST00000317058,sense,AZD_200nM,2.779515
3,11.0,241.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,CCDC101,65.657931,-32.481356,10.267112,-50.588522,0,ENST00000317058,antisense,AZD_200nM,2.301829
4,13.0,216.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,69.998236,-38.581712,17.927823,-10.007838,0,ENST00000317058,sense,AZD_200nM,2.255644
5,12.0,244.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,65.261154,-33.479130,19.039549,-25.595133,0,ENST00000317058,antisense,AZD_200nM,2.115588
6,10.0,78.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,62.772844,-26.148099,4.165713,-30.401277,0,ENST00000317058,antisense,AZD_200nM,2.066618
7,11.0,180.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,67.936664,-32.704164,7.335508,-25.785900,0,ENST00000317058,antisense,AZD_200nM,2.040182
8,13.0,116.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,CCDC101,70.542243,-34.839040,21.391726,-39.048828,0,ENST00000317058,sense,AZD_200nM,2.025409
9,10.0,278.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,CCDC101,64.483011,-44.946409,12.161792,-32.602279,0,ENST00000317058,sense,AZD_200nM,2.002125


In [4]:
dummy_df = pd.get_dummies(df).dropna(axis=0)
dummy_df.shape

(12984, 4514)

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            12977, 12978, 12979, 12980, 12981, 12982, 12983, 12984, 12985,
            12986],
           dtype='int64', length=12984)

In [5]:
cols = dummy_df.columns.tolist()
cols.append(cols.pop(cols.index('lfc')))
dummy_df = dummy_df.reindex(columns= cols)

In [6]:
dummy_df.columns[-1]

'lfc'

In [7]:
X, y = dummy_df.iloc[:,:-1].values, dummy_df.iloc[:,-1].values

In [58]:
model = GradientBoostingRegressor()
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
sum(abs(cross_val))/len(cross_val)

array([-2.51362872, -2.83527368, -2.50070136, -5.03231367, -1.77300729])

In [61]:
-sum([-2.51362872, -2.83527368, -2.50070136, -5.03231367, -1.77300729])/5

2.930984944

In [60]:
model = xg.XGBRegressor()
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")

-2.717733243254866

In [62]:
-sum(cross_val)/len(cross_val)

2.717733243254866

In [72]:
2.930984944/2.3380535190836502

1.2536004501508315

In [70]:
model = xg.XGBRegressor(n_estimators=1000)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")

In [71]:
-sum(cross_val)/len(cross_val)

2.3380535190836502

In [44]:
[i for i,j in enumerate(dummy_df.columns.values) if j == "lfc"]

[149]

In [9]:
model = RandomForestRegressor(bootstrap=True, 
                              max_features=0.25, 
                              min_samples_leaf=3, 
                              min_samples_split=4,
                              n_estimators=100)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
sum(abs(cross_val))/len(cross_val)

2.7437113489091631

In [ ]:
model = LassoLarsCV(normalize=True)
cross_val = cross_validation.cross_val_score(estimator=model, 
                                             X=X, 
                                             y=y, 
                                             cv = 5, 
                                             scoring="mean_squared_error",
                                             n_jobs=4)
sum(abs(cross_val))/len(cross_val)